In [ ]:
import numpy as np
try:
    from docplex.mp.model import Model
except:
    !pip install docplex
from docplex.mp.model import Model

import pandas as pd

try:
    import ipywidgets
except:
    !pip install ipywidgets
from ipywidgets import interact
import ipywidgets as widgets
import time

# Cotas de restricciones

In [ ]:
# Numero de enfermeras
N = 15 #dato arbitrario
nurses = ['Enfermera_' +str(n) for n in range(N)]
# periodo de días que queremos agendar
T = 7 #una semana
days = ['Dia_' +str(t) for t in range(T)]
# Turnos asumiendo 3 tipos de turnos
S = ['Mañana', 'Tarde', 'Noche']
# numero de enferrmeras requerido por turno
R = {'Mañana' : 5,
     'Tarde' : 4,
     'Noche' : 3}
# Pesos asignados a cada turno
W = {'Mañana' : 10,
     'Tarde' : 10,
     'Noche' : 10}

In [ ]:
# Variables temporales que podrían arreglar el trabajo de ciertos datos
days2 = ['Dia_' +str(t+1) for t in range(T-1)]
days3 = ['Dia_' +str(t) for t in range(6)]

In [ ]:
mdl = Model('Scheduling')
# Creación de variables
idx_x = [(i,s,t) for i in nurses for s in S for t in days]
idd=[(s) for s in S]
x = mdl.binary_var_dict(idx_x) #Diccionario con días, nombre de personal y turno correspondiente
D_dif = mdl.continuous_var #Diferencia entre el máximo y mínimo de turnos de día trabajados
E_dif = mdl.continuous_var #Diferencia entre el máximo y mínimo de turnos de tarde trabajados
N_dif = mdl.continuous_var #Diferencia entre el máximo y mínimo de turnos nocturnos trabajados
O_dif = mdl.continuous_var #Diferencia entre el máximo y mínimo de días libres
mini = mdl.continuous_var_dict(idd) #Numero mínimo de turnos s asignados a las enfermeras en el calendario
maxi = mdl.continuous_var_dict(idd) #Numero máximo de turnos s asignados a las enfermeras en el calendario

# Función objetivo

\begin{equation*}
\begin{aligned}
& \underset{x_{ist}\in\{0,1\}}{\text{min}}(\sum _{s \in S }(( \underset{i \in 0,1,\ldots , N}{\max}\sum_{t=1}^T x_{ist}-\underset{i \in 0,1,\ldots , N}{\min}\sum_{t=1}^T x_{ist})\cdot W(s)))\\
\end{aligned}
\end{equation*}

In [ ]:
mdl.minimize(mdl.sum(maxi[s]-mini[s]*W[s] for s in S)) #Intento tipo 1 minimizar diferencia entre maximos y minimos con peso
#mdl.minimize(mdl.sum(x[i,s,t]*W[s] for i in nurses for s in S for t in days))#Intento tipo 1 minimizar diferencia entre maximos y minimos con peso

# Restricciones

In [ ]:
#El perrsonal no puede tener más de un turno al día
mdl.add_constraints(mdl.sum(x[i,s,t] for s in S) <= 1 for i in nurses for t in days);
#Se cumple con tener el personal asignado para cada turno del día
mdl.add_constraints(mdl.sum(x[i,s,t] for i in nurses)>= R[s]  for s in S for t in days);
#No se trabaja un día si se trabaja una noche el día anterior.
mdl.add_constraints(mdl.sum(x[i,'Mañana',t] for i in nurses for t in days)-1 <= x[i,'Noche',t] for i in nurses for t in days2);
#Cada integrante del personal no trabaja más de una cantidad especifica de noches
mdl.add_constraints( mdl.sum(x[i,'Noche',t] for i in nurses)<= 6 for t in days ); #Falta implementar que no sea en días consecutivos

In [ ]:
mdl.print_information()
mdl.solve()
mdl.solution.solve_details #Codigo inconcluso.

In [ ]:
mdl.report()

In [ ]:
status = mdl.solve_details.status == 'integer optimal solution'
status

## Analysing the solution

Below we provide simple tools to analyse the solution obtained

In [ ]:
def x_star_to_pandas(x):
    '''
    takes in input the solution of the optimization problem as a dictionary 
    returns the solution as a dataframe 
    '''
    sol = pd.DataFrame(columns = ['Nurse', 'Shift', 'Day'])
    k = 0
    for key, value in x.items():
        if value>0:
            sol.loc[k] =np.array([i for i in key])
            k+=1
    return sol

In [ ]:
# transform the solution into a dataframe
x_star_dict =mdl.solution.get_value_dict(x)
sol_x = x_star_to_pandas(x_star_dict)
sol_x.head()

### Visualization tool

Below we provide a tool to check the schedule. 

In [ ]:
# remove warning from pandas (in the viz_tool it does what we need)
import warnings
warnings.simplefilter(action='ignore')

In [ ]:

def viz_tool(nurse,shift,day):
    '''
    interactive function to extract the information required:
    if a value is 'All' then it returns all the values for that specific feature
    '''
    global nurses,S,days
    
    if nurse == 'All':
        df_tmp = sol_x[(sol_x['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_x[(sol_x['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_x['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_x['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_x['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_x['Day']==day)]

    print(df_tmp)

interact(viz_tool, nurse = widgets.Dropdown(value="All",placeholder='Type something', options=nurses+['All']),
              shift=widgets.Dropdown(value='All',placeholder='Type something', options=S+['All']),
              day = widgets.Dropdown(value="All",placeholder='Type something', options=days+['All'])
        );
